In [1]:
import transformers


# use sguskin/minilmv2-L6-H384-squad1.1
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("sguskin/minilmv2-L6-H384-squad1.1")
model = AutoModelForQuestionAnswering.from_pretrained("sguskin/minilmv2-L6-H384-squad1.1")


In [2]:
# reading from dev-v1.1.json
import json
data = json.load(open("./dev-v1.1.json"))

In [3]:
import time
from tqdm import tqdm
import torch

def parse_data(data):
    # parses data to only retrieve id, context, question
    questions = []
    contexts = []
    ids = []
    for data_item in data["data"]:
        for paragraph in data_item["paragraphs"]:
            for qa in paragraph["qas"]:
                questions.append(qa["question"])
                contexts.append(paragraph["context"])
                ids.append(qa["id"])
    
    return zip(ids, questions, contexts)

def answer(model, data, output_file):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    start_time = time.time()
    res = {}
    for id_, question, context in tqdm(parse_data(data)):
        inputs = tokenizer.encode_plus(question, context,
                                       add_special_tokens=True, return_tensors="pt",
                                       max_length=512, truncation='only_second', padding='max_length'
                                       ).to(device)
        
        input_ids = inputs["input_ids"].tolist()[0]

        text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
        answer_start_scores, answer_end_scores = model(**inputs, return_dict=False)
        answer_start = torch.argmax(answer_start_scores).to(device)
        answer_end = torch.argmax(answer_end_scores).to(device) 
    
        predict_anser_tokens = text_tokens[answer_start:answer_end+1]
        predict_answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(predict_anser_tokens))
        res[id_] = predict_answer
        
    print("Time taken: ", time.time() - start_time, "seconds")
    
    with open(output_file, 'w') as f:
        json.dump(res, f)

    print("Done writing to file")
            
answer(model, data, "predictions_minimlv.json")

0it [00:00, ?it/s]2023-11-13 14:20:09.539321: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-13 14:20:09.539357: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
10570it [00:35, 296.00it/s]

Time taken:  35.71459221839905 seconds
Done writing to file


In [4]:
!python evaluate-v2.0.py dev-v1.1.json predictions_minimlv.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{
  "exact": 79.78240302743615,
  "f1": 87.21905198690425,
  "total": 10570,
  "HasAns_exact": 79.78240302743615,
  "HasAns_f1": 87.21905198690425,
  "HasAns_total": 10570
}
